# Content-Based

In [31]:
import pandas as pd
import numpy as np

In [8]:
ratings = pd.read_csv('ratings_p.csv')
genres = pd.read_csv('genres_d.csv')

In [9]:
# 1번 유저 데이터 따로 떼어내 테이블 만들기
user1 = ratings[ratings['userId']==1]
user1.shape

(232, 6)

In [16]:
user1 = user1.sort_values('movieId')
user1.head()

,userId,movieId,rating,title,genres,year
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,1995
2,1,6,4.0,Heat (1995),Action|Crime|Thriller,1995
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995


In [17]:
genres.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
# genres에서 1번 유저가 평점 준 영화 데이터만 가져오기
user1_movies = genres[genres.index.isin(user1['movieId'])]

In [19]:
user1_movies.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
47,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
50,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [20]:
user1['rating'].head()

0    4.0
1    4.0
2    4.0
3    5.0
4    5.0
Name: rating, dtype: float64

In [23]:
user1_movies.shape

(232, 20)

In [51]:
user1_movies.T

,1,3,6,47,50,70,101,110,151,157,...,3671,3702,3703,3729,3740,3744,3793,3809,4006,5060
(no genres listed),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Action,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Adventure,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Animation,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Children,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comedy,0,1,1,1,0,0,0,0,0,1,...,0,0,0,1,1,1,0,0,0,0
Crime,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
Documentary,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
Drama,0,1,0,1,1,1,0,0,1,0,...,1,1,1,0,0,0,1,1,0,0
Fantasy,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [59]:
user1_movies.T.dot(user1['rating'])

ValueError: matrices are not aligned

In [74]:
np.dot(user1_movies.T, user1['rating'])

array([  0., 159., 110.,  44.,  78., 284., 116.,  25., 470.,  56.,  14.,
       130.,   0.,  34.,  67., 177.,  73., 211.,  44.,   6.])